In [48]:
import os
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from gensim.models.word2vec import Word2Vec
import nltk

In [51]:
raw_text = ''
for file in os.listdir("."):
    if file.endswith(".txt"):
        raw_text += open(file, errors = 'ignore').read()+'\n\n'

raw_text = raw_text.lower()

In [52]:
raw_text[:500]

'the project gutenberg ebook, what the schools teach and might teach, by\njohn franklin bobbitt\n\n\nthis ebook is for the use of anyone anywhere at no cost and with\nalmost no restrictions whatsoever.  you may copy it, give it away or\nre-use it under the terms of the project gutenberg license included\nwith this ebook or online at www.gutenberg.net\n\n\n\n\n\ntitle: what the schools teach and might teach\n\nauthor: john franklin bobbitt\n\nrelease date: september 16, 2004  [ebook #13482]\n\nlanguage: english\n\ncha'

In [53]:
sentensor = nltk.data.load('tokenizers/punkt/english.pickle')

In [54]:
sents = sentensor.tokenize(raw_text)
corpus = []
for sentence in sents:
    corpus.append(nltk.word_tokenize(sentence))

In [55]:
print(len(corpus))
print(corpus[:3])

17700
[['the', 'project', 'gutenberg', 'ebook', ',', 'what', 'the', 'schools', 'teach', 'and', 'might', 'teach', ',', 'by', 'john', 'franklin', 'bobbitt', 'this', 'ebook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever', '.'], ['you', 'may', 'copy', 'it', ',', 'give', 'it', 'away', 'or', 're-use', 'it', 'under', 'the', 'terms', 'of', 'the', 'project', 'gutenberg', 'license', 'included', 'with', 'this', 'ebook', 'or', 'online', 'at', 'www.gutenberg.net', 'title', ':', 'what', 'the', 'schools', 'teach', 'and', 'might', 'teach', 'author', ':', 'john', 'franklin', 'bobbitt', 'release', 'date', ':', 'september', '16', ',', '2004', '[', 'ebook', '#', '13482', ']', 'language', ':', 'english', 'character', 'set', 'encoding', ':', 'iso-646-us', '(', 'us-ascii', ')', '***start', 'of', 'the', 'project', 'gutenberg', 'ebook', 'what', 'the', 'schools', 'teach', 'and', 'might', 'teach***', 'e-text', 'prepared', 'b

In [56]:
w2v_model = Word2Vec(corpus, size=128, window=5, min_count=5, workers=4)

In [106]:
w2v_model['education'],type(w2v_model)

/home/dayou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


(array([ 0.02310123,  0.14339939, -0.01574001, -0.14812136,  0.58821553,
        -0.21386705,  0.25770804, -0.10564516,  0.21918327,  0.09389284,
        -0.11073962, -0.19918956,  0.27232224,  0.11480273,  0.04575203,
        -0.03868816, -0.04713798, -0.0156972 ,  0.06937253,  0.01929595,
         0.37166324, -0.02945311, -0.15907137, -0.12076304, -0.10689121,
         0.6203583 ,  0.03230147, -0.03147005, -0.01747278,  0.36145404,
        -0.14510497,  0.4807741 , -0.12574878,  0.19853745,  0.176683  ,
        -0.03961089,  0.11874843,  0.19344407,  0.07168339, -0.06525571,
         0.04289619, -0.14397055,  0.10810485, -0.17908727, -0.08712856,
         0.2100108 ,  0.08310189, -0.09134321,  0.11114113, -0.19586925,
        -0.18574558,  0.20928854,  0.06638116,  0.19759011,  0.04168693,
        -0.02232549, -0.02697749, -0.17882301,  0.10315129,  0.09207642,
        -0.01588452, -0.08671781, -0.09527508, -0.20701055,  0.10484719,
         0.02010195,  0.02898908, -0.04954598,  0.2

In [58]:
raw_input = [term for sublist in corpus for term in sublist]

In [59]:
len(raw_input),type(w2v_model)

(416566, gensim.models.word2vec.Word2Vec)

In [60]:
raw_input[18]

'ebook'

In [61]:
text_stream = []
vocab = w2v_model.wv.vocab
for word in raw_input:
    if word in vocab:
        text_stream.append(word)
len(text_stream)

393290

In [62]:
type(vocab),len(vocab),type(text_stream)

(dict, 5941, list)

In [63]:
vocab['education']

In [64]:
seq_length = 10
x = []
y = []
for i in range(0, len(text_stream)-seq_length):
    in_words = text_stream[i:i+seq_length]
    out_word = text_stream[i+seq_length]
    x.append([w2v_model[word] for word in in_words])
    y.append(w2v_model[out_word])

/home/dayou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys
/home/dayou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [65]:
len(x),type(x),len(x[0][0])

(393280, list, 128)

In [66]:
print(x[10])
print(y[10])

[array([-0.3905075 ,  0.03278019, -0.19273032, -0.4317037 ,  0.25757724,
       -0.09018032,  0.6704407 ,  0.20358521,  0.19320281,  0.08482135,
       -0.10894309,  0.8739892 ,  0.32603928,  1.0417536 , -0.1960928 ,
       -0.19402346, -0.36094838, -0.34184387, -0.3239454 , -0.31998703,
        1.1243871 , -0.25620934, -0.5914936 ,  0.2935557 ,  0.32723173,
        1.349506  , -0.31665304,  0.2180652 , -0.01265861,  0.17446776,
       -0.768158  ,  0.24424805, -0.35331365,  0.23031715, -0.19081062,
       -0.71738106, -0.13373011,  0.21995184, -0.29008326, -0.00917233,
        0.5144211 , -0.15074936,  0.57395345, -0.26303777, -0.2153222 ,
        0.24226162,  0.3829825 , -0.33512753, -0.5680728 , -0.43565437,
       -0.3604242 ,  0.6401822 ,  0.28843492, -0.0161855 , -0.6852268 ,
       -0.25418508,  0.26278922,  0.21840239,  0.2261932 ,  0.7408421 ,
        0.48652944,  0.3283992 , -0.13891336, -0.56153935, -0.019992  ,
       -0.02120919,  0.44033703,  0.23172316,  0.82097477, -0.3

In [68]:
x = np.reshape(x, (-1, seq_length, 128))
y = np.reshape(y, (-1, 128))

In [69]:
model = Sequential()
model.add(LSTM(256, dropout=0.2, input_shape=(seq_length,128)))
model.add(Dropout(0.2))
model.add(Dense(128, activation='sigmoid'))

In [70]:
model.compile(loss='mse', optimizer='adam')

In [117]:
model.fit(x, y, epochs=10, batch_size=128)

Epoch 1/10
393280/393280 [==============================] - 956s - loss: 0.1562   
Epoch 2/10
393280/393280 [==============================] - 1116s - loss: 0.1548  
Epoch 3/10
393280/393280 [==============================] - 1117s - loss: 0.1540  
Epoch 4/10
393280/393280 [==============================] - 1190s - loss: 0.1532  
Epoch 5/10
393280/393280 [==============================] - 2393s - loss: 0.1525  
Epoch 6/10
393280/393280 [==============================] - 1144s - loss: 0.1520  
Epoch 7/10
393280/393280 [==============================] - 1110s - loss: 0.1515  
Epoch 8/10
393280/393280 [==============================] - 1111s - loss: 0.1512  
Epoch 9/10
393280/393280 [==============================] - 1111s - loss: 0.1508  
Epoch 10/10
393280/393280 [==============================] - 1112s - loss: 0.1506  


In [118]:
def predict_next(input_array):
    x = np.reshape(input_array, (-1, seq_length, 128))
    y = model.predict(x)
    return y

def string_to_index(raw_input):
    raw_input = raw_input.lower()
    input_stream = nltk.word_tokenize(raw_input)
    
    res=[]
    for c in input_stream[(len(input_stream) - seq_length):]:
        res.append(w2v_model[c])
    return res

def y_to_word(y): 
    word = w2v_model.most_similar(y, topn=1)
    return word

In [119]:
def generate_article(init, num=20):
    in_string = init.lower()
    for i in range(num):
        print(in_string)
        res = string_to_index(in_string)
        #print(res)
        y = predict_next(res)
        #print(y)
        n = y_to_word(y)
        in_string += ' ' + n[0][0]
    return in_string

In [125]:
init = 'slain and toasted the night before,and drank some water from a large puddle'

out = generate_article(init)

print(out)

slain and toasted the night before,and drank some water from a large puddle


/home/dayou/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


KeyError: "word 'puddle' not in vocabulary"